# Business Probelm
If you choose the Recommendation System option, you will be making movie recommendations based on the MovieLensLinks to an external site. dataset from the GroupLens research lab at the University of Minnesota. Unless you are planning to run your analysis on a paid cloud platform, we recommend that you use the "small" dataset containing 100,000 user ratings (and potentially, only a particular subset of that dataset).

Your task is to:

Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.
The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".

Collaborative Filtering
At minimum, your recommendation system must use collaborative filtering. If you have time, consider implementing a hybrid approach, e.g. using collaborative filtering as the primary mechanism, but using content-based filtering to address the cold start problemLinks to an external site..

Evaluation
The MovieLens dataset has explicit ratings, so achieving some sort of evaluation of your model is simple enough. But you should give some thought to the question of metrics. Since the rankings are ordinal, we know we can treat this like a regression problem. But when it comes to regression metrics there are several choices: RMSE, MAE, etc. HereLinks to an external site. are some further ideas.

In [12]:

import numpy as np
import pandas as pd

In [13]:
# Reading in datasets
df_movies = pd.read_csv('data/movies.csv')
df_links = pd.read_csv('data/links.csv')
df_ratings = pd.read_csv('data/ratings.csv')
df_tags = pd.read_csv('data/tags.csv')

In [14]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [15]:
df_links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [19]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [18]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [20]:
#Importing Libraries
df = pd.concat(
    map(pd.read_csv, ['data/movies.csv','data/ratings.csv', 'data/tags.csv']), ignore_index=True)
df

,movieId,title,genres,userId,rating,timestamp,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
114256,7382,NaN,NaN,606.0,NaN,1.171234e+09,for katie
114257,7936,NaN,NaN,606.0,NaN,1.173392e+09,austere
114258,3265,NaN,NaN,610.0,NaN,1.493844e+09,gun fu
114259,3265,NaN,NaN,610.0,NaN,1.493844e+09,heroic bloodshed


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114261 entries, 0 to 114260
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    114261 non-null  int64  
 1   title      9742 non-null    object 
 2   genres     9742 non-null    object 
 3   userId     104519 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  104519 non-null  float64
 6   tag        3683 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 6.1+ MB


In [6]:
df['rating'].value_counts()
# ratings range from .5 - 4.

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [7]:
df['genres'].value_counts()

Drama                                      1053
Comedy                                      946
Comedy|Drama                                435
Comedy|Romance                              363
Drama|Romance                               349
                                           ... 
Action|Adventure|Fantasy|War                  1
Action|Adventure|Children|Comedy|Crime        1
Adventure|Animation|Fantasy|IMAX              1
Adventure|Children|Drama|Fantasy              1
Adventure|Drama|Horror|Mystery|Thriller       1
Name: genres, Length: 951, dtype: int64

In [8]:
df['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Bungo Stray Dogs: Dead Apple (2018)',
       'Andrew Dice Clay: Dice Rules (1991)', nan], dtype=object)

In [9]:
df.title.value_counts()

Eros (2004)                               2
Emma (1996)                               2
War of the Worlds (2005)                  2
Saturn 3 (1980)                           2
Confessions of a Dangerous Mind (2002)    2
                                         ..
Wild Bill (1995)                          1
Peyton Place (1957)                       1
Smokin' Aces (2006)                       1
Finding Dory (2016)                       1
Blame It on the Bellboy (1992)            1
Name: title, Length: 9737, dtype: int64

In [10]:
df.head()

,movieId,title,genres,userId,rating,timestamp,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN,NaN,NaN,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114261 entries, 0 to 114260
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    114261 non-null  int64  
 1   title      9742 non-null    object 
 2   genres     9742 non-null    object 
 3   userId     104519 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  104519 non-null  float64
 6   tag        3683 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 6.1+ MB
